# Volume 3: Discrete Hidden Markov Models
    <Name>
    <Class>
    <Date>

In [1]:
import numpy as np
import copy
import string
import codecs
#from hmmlearn import hmm

# Problems 1-5
This is the HMM class that you will be adding functions to throughout the lab.

In [19]:
class HMM(object):
    """
    Finite state space hidden Markov model.
    """
    
    # Problem 1
    def __init__(self, A, B, pi):
        """
        Initialize an HMM with parameters A, B, and pi.
        """
        # initialize
        self.A = A
        self.B = B
        self.pi = pi
    
    # Problem 2
    def forward_pass(self, z):
        """
        Compute the forward probability matrix.

        Parameters
        ----------
        z : ndarray of shape (T,)
            The observation sequence

        Returns
        -------
        alpha : ndarray of shape (T, n)
            The forward probability matrix
        """
        # initialize
        T = len(z)  # observation sequence length
        n = self.A.shape[0]  # number of states
        alpha = np.zeros((T, n))  # forward probability matrix

        # compute alpha[0] for all states 
        alpha[0] = self.pi * self.B[z[0], :]

        # calculate alpha[t] for t > 0
        for t in range(1, T):
            for i in range(n):
                for j in range(n):
                    alpha[t, i] += alpha[t-1, j] * self.A[i, j]
                alpha[t, i] *= self.B[z[t], i] 

        return alpha
    
    # Problem 4
    def backward_pass(self, z):
        """
        Compute the backward probability matrix and gamma values.

        Parameters
        ----------
        z : ndarray of shape (T,)
            The observation sequence

        Returns
        -------
        beta : ndarray of shape (T, n)
            The backward probability matrix
        gamma : ndarray of shape (T, n)
            The state probability matrix
        """
        # initialize
        T = len(z)  # observation sequence length
        n = self.A.shape[0]  # number of states
        beta = np.zeros((T, n))  # backward probability matrix
        beta[-1] = 1
        gamma = np.zeros((T, n))  # state probability matrix

        # calculate P(z|θ) using forward pass
        alpha = self.forward_pass(z)
        P_z_theta = np.sum(alpha[-1,:])        

        # recursive step computing beta[t] for t < T-1
        for t in range(T-2, -1, -1):
            for j in range(n):
                for i in range(n):
                    beta[t, j] += self.A[i, j] * beta[t+1, i] * self.B[z[t+1], i] 
                    
        # compute gamma values
        for t in range(T):
            for i in range(n):
                gamma[t, i] = alpha[t, i] * beta[t, i] / P_z_theta
                
        return beta, gamma
    
    # Problem 5
    def viterbi_algorithm(self, z):
        """
        Compute the most likely hidden state sequence using the Viterbi algorithm.

        Parameters
        ----------
        z : ndarray of shape (T,)
            The observation sequence

        Returns
        -------
        x*: ndarray of shape (T,)
            The most likely state sequence
        """
        # initialize
        T = len(z)  # observation sequence length
        n = self.A.shape[0]  # number of states
        xstar = np.zeros(T, dtype=int)  # most likely state sequence

        eta = np.zeros((T, n))
        for i in range(n):
            eta[0, i] = self.B[z[0], i] * self.pi[i]
        
        for t in range(1, T):
            for i in range(n):
                eta[t, i] = np.max(self.B[z[t],i] * self.A[:,i] * eta[t-1])
        
        xstar[-1] = np.argmax(eta[-1])
        
        for t in range(T-2, -1, -1):
            xstar[t] = np.argmax(eta[t,:] * self.A[xstar[t+1], :])
            
        return xstar



# Problem 2 test case

Use the following HMM and code to test your HMM class.
Compare the output to `forward_pass` with the lab pdf.

In [20]:
pi = np.array([.6, .4])
A = np.array([[.7, .4],[.3, .6]])
B = np.array([[.1,.7],[.4, .2],[.5, .1]])
z_example = np.array([0, 1, 0, 2])
example_hmm = HMM(A, B, pi)

alpha=example_hmm.forward_pass(z_example)
print(np.sum(alpha[-1,:]))

0.009629599999999997


# Problem 3

Consider the following (very simplified) model of the price of a stock over time as an HMM.
The observation states will be the change in the value of the stock.
For simplicity, we will group these into five values: large decrease, small decrease, no change, small increase, large increase, labeled as integers from 0 to 4.
The hidden state will be the overall trends of the market.
We'll consider the market to have three possible states: declining in value (bear market), not changing in value (stagnant), and increasing in value (bull market), labeled as integers from 0 to 2.
Let the HMM modeling this scenario have parameters
$$
\boldsymbol\pi=\begin{bmatrix}
1/3 \\ 1/3 \\ 1/3
\end{bmatrix},
\quad
A=\begin{bmatrix}
0.5 & 0.3 & 0 \\
0.5 & 0.3 & 0.3 \\
0 & 0.4 & 0.7
\end{bmatrix},
\quad
B=\begin{bmatrix}
0.3 & 0.1 & 0 \\
0.3 & 0.2 & 0.1 \\
0.3 & 0.4 & 0.3 \\
0.1 & 0.2 & 0.4 \\
0 & 0.1 & 0.2
\end{bmatrix}
$$
The file `stocks.npy` contains a sequence of 50 observations drawn from this HMM.
What is the probability of this observation sequence given these model parameters?
Use your implementation of the forward pass algorithm from Problem 2 to find the answer.
Note that the answer is very small, because there are lots of possible observation sequences.

In [21]:
# HMM parameter setup
pi = np.array([1/3, 1/3, 1/3])
A = np.array([
    [0.5, 0.3, 0.0],
    [0.5, 0.3, 0.3],
    [0.0, 0.4, 0.7]
])
B = np.array([
    [0.3, 0.1, 0.0],
    [0.3, 0.2, 0.1],
    [0.3, 0.4, 0.3],
    [0.1, 0.2, 0.4],
    [0.0, 0.1, 0.2]
])

In [22]:
# load data
z = np.load('stocks.npy')
# build model
stock_hmm = HMM(A, B, pi)
prob = stock_hmm.forward_pass(z)[-1].sum()
print(prob)

6.671115114537777e-34


# Problem 4

Create a method `backward_pass` in your HMM class to implement the backward pass algorithm.
This function should accept the observation sequence $\mathbf{z}$ and return two arrays of the $\beta_t(i)$ and $\gamma_t(i)$ values.

Test your function on the example HMM, and compare the output with the lab pdf.

With your function and the stock model from Problem 3, answer the following question: given the observation sequence in `stocks.npy`, what is the most likely initial hidden state $X_0$?

In [23]:
# Test case; compare your output with what is in the lab pdf
beta, gamma = example_hmm.backward_pass(z_example)
print(beta)
print(gamma)

[[0.0302  0.02792]
 [0.0812  0.1244 ]
 [0.38    0.26   ]
 [1.      1.     ]]
[[0.18816981 0.81183019]
 [0.51943175 0.48056825]
 [0.22887763 0.77112237]
 [0.8039794  0.1960206 ]]


In [24]:
beta, gamma = stock_hmm.backward_pass(z)
print(gamma[0])
# print the most likely class
print(np.argmax(gamma[0]))

[0.39109931 0.39359405 0.21530664]
1


# Problem 5
Creating a method `viterbi_algorithm` in your HMM class to implement the Viterbi algorithm.
This function should accept the observation sequence $\mathbf{z}$ and return the most likely state sequence $\mathbf{x}^*$.

Test your function on the example HMM and compare output with the lab pdf.

Apply your function to the stock market HMM from Problem 3.
With the observaition sequence from `stocks.npy`, what is the most likely sequence of hidden states?
Is the initial state of the most likely sequence the same as the most likely initial state you found in Problem 4?

In [25]:
# Test case
xstar = example_hmm.viterbi_algorithm(z_example)
print(xstar)

[1 1 1 0]


In [26]:
xstar = stock_hmm.viterbi_algorithm(z)
print(xstar)

[0 0 0 0 0 1 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 1 0 0 0 1 2 2 2 2 2
 2 2 2 2 2 2 1 0 0 0 0 0 0]


The most likely initialize state is 0 instead of 1.

# Problem 6

Train a `hmmlearn.hmm.CategoricalHMM` on `declaration.txt`. Use `N=2` states and `M=len(set(obs))=27` observation values (26 lower case characters and 1 whitespace character).
Use `n_iter=200` and `tol=1e-4`.

Once the learning algorithm converges, analyze the state observation matrix $B$. Note which rows correspond to the largest and smallest probability values in each column of $B$, and check the corresponding characters. The HMM should have detected a vowel state and a consonant state.

In [34]:
def vec_translate(a, my_dict):
    # translate numpy array from symbols to state numbers or vice versa
    return np.vectorize(my_dict.__getitem__)(a)

def prep_data(filename):
    """
    Reads in the file and prepares it for use in an HMM.
    Returns:
        symbols (dict): a dictionary that maps characters to their integer values
        obs_sequence (ndarray): an array of integers representing the read-in text
    """
    # Get the data as a single string
    with codecs.open(filename, encoding='utf-8') as f:
        data=f.read().lower()  # and convert to all lower case
    # remove punctuation and newlines
    remove_punct_map = {ord(char): 
                        None for char in string.punctuation+"\n\r"}
    data = data.translate(remove_punct_map)
    # make a list of the symbols in the data
    symbols = sorted(list(set(data)))
    # convert the data to a NumPy array of symbols
    a = np.array(list(data))
    # make a conversion dictionary from symbols to state numbers
    symbols_to_obsstates = {x:i for i,x in enumerate(symbols)}
    # convert the symbols in a to state numbers
    obs_sequence = vec_translate(a,symbols_to_obsstates)
    return symbols, obs_sequence

In [35]:
symbols, obs = prep_data('declaration.txt')

In [38]:
from hmmlearn import hmm

h = hmm.CategoricalHMM(n_components=2, n_features = 27, tol=1e-4, n_iter=200, )
h.fit(obs.reshape(-1,1))

pi = h.startprob_
A = h.transmat_.T
B = h.emissionprob_.T

for i in range(len(B)):
    print(f"{symbols[i]}, {B[i, 0]:0.4f}, {B[i, 1]:0.4f}")


 , 0.1535, 0.1866
a, 0.0619, 0.0592
b, 0.0000, 0.0347
c, 0.0301, 0.0112
d, 0.0495, 0.0001
e, 0.1306, 0.0710
f, 0.0000, 0.0657
g, 0.0256, 0.0000
h, 0.0686, 0.0000
i, 0.0883, 0.0000
j, 0.0000, 0.0058
k, 0.0028, 0.0000
l, 0.0035, 0.0767
m, 0.0152, 0.0244
n, 0.0950, 0.0000
o, 0.0304, 0.1307
p, 0.0000, 0.0503
q, 0.0000, 0.0022
r, 0.0234, 0.1117
s, 0.0684, 0.0474
t, 0.1156, 0.0191
u, 0.0053, 0.0664
v, 0.0146, 0.0000
w, 0.0156, 0.0065
x, 0.0014, 0.0006
y, 0.0000, 0.0295
z, 0.0008, 0.0000


# Problem 7

Repeat the same calculation with `WarAndPeace.txt` with 2 hidden states. Interpret/explain your results. Which Cyrillic characters appear to be vowels?

In [39]:
symbols, obs = prep_data('WarAndPeace.txt')

In [41]:
h = hmm.CategoricalHMM(n_components=2, tol=1e-4, n_iter=200, )
h.fit(obs.reshape(-1,1))

pi = h.startprob_
A = h.transmat_.T
B = h.emissionprob_.T

for i in range(len(B)):
    print(f"{symbols[i]}, {B[i, 0]:0.4f}, {B[i, 1]:0.4f}")

 , 0.1711, 0.1608
а, 0.1902, 0.0169
б, 0.0000, 0.0217
в, 0.0288, 0.0440
г, 0.0044, 0.0237
д, 0.0087, 0.0296
е, 0.0000, 0.0980
ж, 0.0047, 0.0100
з, 0.0492, 0.0000
и, 0.0731, 0.0448
й, 0.0000, 0.0130
к, 0.0731, 0.0112
л, 0.0685, 0.0319
м, 0.0100, 0.0286
н, 0.0795, 0.0491
о, 0.0000, 0.1390
п, 0.0026, 0.0325
р, 0.0144, 0.0454
с, 0.0598, 0.0341
т, 0.0267, 0.0558
у, 0.0111, 0.0292
ф, 0.0039, 0.0000
х, 0.0000, 0.0096
ц, 0.0038, 0.0026
ч, 0.0058, 0.0142
ш, 0.0053, 0.0071
щ, 0.0012, 0.0036
ъ, 0.0000, 0.0004
ы, 0.0000, 0.0217
ь, 0.0321, 0.0115
э, 0.0000, 0.0038
ю, 0.0046, 0.0062
я, 0.0675, 0.0000
ё, 0.0000, 0.0001


Based on the data provided, the Cyrillic characters а, е, и, о, у, ы, э, ю, я, and ё show as vowels. 